In [1]:
import torchvision
import torch
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import ProjectedGradientDescent
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from torchvision import transforms
from torchvision.io import read_image
from art.estimators.classification import PyTorchClassifier
# from art.attacks.evasion import ProjectedGradientDescentPyTorch
from torchattacks import PGD
from art.utils import random_sphere
from art.config import ART_NUMPY_DTYPE
import numpy as np
from torchvision.transforms import v2
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader
import torchvision.datasets as datasets
from torch.utils.data.dataset import Dataset
import torch.nn as nn
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights, vgg19 , VGG19_Weights, resnet18, ResNet18_Weights, densenet121,  densenet121, DenseNet121_Weights, vit_b_16, ViT_B_16_Weights, efficientnet_b2, EfficientNet_B2_Weights,  swin_s, Swin_S_Weights, convnext_base, ConvNeXt_Base_Weights

/home/baravrah/.conda/envs/gpu_test/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/baravrah/.conda/envs/gpu_test/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/baravrah/.conda/envs/gpu_test/lib/python3.8/

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
models = [("vit_b_16",vit_b_16, ViT_B_16_Weights),
          ("efficientnet_b0" , efficientnet_b0, EfficientNet_B0_Weights),
          ("swin_s", swin_s, Swin_S_Weights),
          ("densenet121" , densenet121, DenseNet121_Weights),
          ("resnet18" ,resnet18, ResNet18_Weights)]

In [4]:
class model_with_normalization(nn.Module):
    def __init__(self, model, normalization):
        super(model_with_normalization, self).__init__()
        self.model = model 
        self.normalization = normalization
        
    def forward(self, x):
        x = self.normalization(x)
        if x.shape == (3,224,224):
          x = x.unsqueeze(0)
        out = self.model(x)
        return out
    
normalization = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])

In [5]:
import os
from torchvision import transforms
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self):
        self.data = []

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Initialize your custom dataset
# new_dataset = CustomDataset()

In [6]:
def read_image(image_path):
    return Image.open(image_path).convert('RGB')  # Ensure the image is in RGB format

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),])

In [7]:
import pickle
import random
from torch.cuda.amp import autocast
trans = [
    transforms.RandomAffine(degrees=(-2, 2), translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05),
    # transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomCrop(size=(224, 224), padding=10),
    transforms.GaussianBlur(kernel_size=3),
    transforms.RandomAdjustSharpness(sharpness_factor=2),
    transforms.RandomAutocontrast()]


augmentations = transforms.Compose(trans)

batch_size = 100

In [8]:
e = 12.75

In [ ]:
#ALGORITHM COMPONENTS
for i , (name_v, model_v, weights_v) in enumerate(models): 
    if i !=0 :
        continue
    victim = model_v(weights=weights_v.DEFAULT)
    print("______________________")
    print("Victim: "+ name_v)

    S = models.copy()
    S.pop(i)
    victim = model_with_normalization(victim, normalization)
    victim.eval()
    victim = victim.to(device)
    
    with open('data_'+name_v+'1000images.pkl', 'rb') as f:
        new_data_loader = pickle.load(f)

    #new_data_loader = DataLoader(new_data_loader.dataset, batch_size=batch_size, shuffle=True)
    results = []
    print(e)
    for j , (name_g,model_g, weights_g) in enumerate(S):
               row = [0,0,0,0,0,0,0,0]
               print("g: "+ name_g) 
               g = model_g(weights=weights_g.DEFAULT) 
               Se = S.copy()
               Se.pop(j)
               g = model_with_normalization(g, normalization)

               g.eval()
               g = g.to(device)
               for batch , (images, labels) in enumerate(new_data_loader):

                   augmentations_confidences = []
                   augmentations_success = []

                   augmentation_confidences = []
                   augmentation_success = [] 

                   augmentations_confidences_attack = []
                   augmentations_success_attack = []

                   augmentation_confidences_attack = []
                   augmentation_success_attack = [] 

                   print(batch)

                   if batch==10:
                        break
                   for _ in tqdm(range(200)):

                        victim = victim.to(device)
                        PGD_attack = PGD(g, eps=e/255, alpha=random.uniform(0.1/255, 0.3/255), steps =random.randint(10, 20), random_start=True)

                        #mixing augmentations
                        x_a = augmentations(images)
                        confidance = victim(x_a).softmax(dim=1)
                        adv_label_a = confidance.argmax(dim=1)  
                        adv_label_a = adv_label_a.to('cpu')


                        x_attack_a = PGD_attack(x_a, labels)
                        attack_label_a = victim(x_attack_a).softmax(dim=1).argmax(dim=1)
                        attack_label_a = attack_label_a.to('cpu')


                        #one augmentation
                        x_o = random.choice(trans)(images)
                        adv_label_o = victim(x_o).argmax(dim=1)
                        adv_label_o = adv_label_o.to('cpu')



                        x_attack_o = PGD_attack(x_a, labels)
                        attack_label_o = victim(x_attack_o).softmax(dim=1).argmax(dim=1)
                        attack_label_o = attack_label_o.to('cpu')

                        torch.cuda.empty_cache()
                        del PGD_attack
                        victim = victim.to('cpu')

                        # average_confidences_p =  np.zeros(batch_size, dtype=np.float32)
                        average_confidences_a =  np.zeros(batch_size, dtype=np.float32)
                        average_confidences_o =  np.zeros(batch_size, dtype=np.float32)


                        average_confidences_attack_a =  np.zeros(batch_size, dtype=np.float32)
                        average_confidences_attack_o =  np.zeros(batch_size, dtype=np.float32)

                        for n , model_s, weights_s in Se:

                            model = model_s(weights=weights_s.DEFAULT)
                            model = model.to(device)
                            model = model_with_normalization(model, normalization)
                            model.eval()



                            outputs_a = model(x_a).softmax(dim=1)
                            confidences_a = outputs_a[torch.arange(outputs_a.size(0)),labels].detach().cpu().numpy()                            
                            average_confidences_a += confidences_a


                            outputs_a = model(x_attack_a).softmax(dim=1)
                            confidences_a = outputs_a[torch.arange(outputs_a.size(0)),labels].detach().cpu().numpy()                            
                            average_confidences_attack_a += confidences_a

                            del outputs_a,confidences_a


                            outputs_o = model(x_o).softmax(dim=1)
                            confidences_o = outputs_o[torch.arange(outputs_o.size(0)),labels].detach().cpu().numpy()                            
                            average_confidences_o += confidences_o

                            outputs_o = model(x_attack_o).softmax(dim=1)
                            confidences_o = outputs_o[torch.arange(outputs_o.size(0)),labels].detach().cpu().numpy()                            
                            average_confidences_attack_o += confidences_o


                            del outputs_o,confidences_o,model



                            torch.cuda.empty_cache()
                        del x_a,x_o,x_attack_o,x_attack_a


                        average_confidences_a /= len(Se)
                        average_confidences_o /= len(Se)
                        equality_tensor_a = (labels != adv_label_a).int() 
                        equality_tensor_o = (labels != adv_label_o).int()

                        average_confidences_attack_a /= len(Se)
                        average_confidences_attack_o /= len(Se)

                        equality_tensor_attack_a = (labels != attack_label_a).int() 
                        equality_tensor_attack_o = (labels != attack_label_o).int() #1 if succsess




                        augmentations_confidences.append(average_confidences_a)
                        augmentations_success.append(equality_tensor_a)
                        augmentation_confidences.append(average_confidences_o)
                        augmentation_success.append(equality_tensor_o)


                        augmentations_confidences_attack.append(average_confidences_attack_a)
                        augmentations_success_attack.append(equality_tensor_attack_a)
                        augmentation_confidences_attack.append(average_confidences_attack_o)
                        augmentation_success_attack.append(equality_tensor_attack_o)

                        torch.cuda.empty_cache()






                   merged_results_a = [tuple(zip(np.array(augmentations_confidences)[:, i], torch.stack(augmentations_success).numpy()[:, i]))
                        for i in range(batch_size)]

                   merged_results_o = [tuple(zip(np.array(augmentation_confidences)[:, i], torch.stack(augmentation_success).numpy()[:, i]))
                        for i in range(batch_size)]



                   merged_results_attack_a = [tuple(zip(np.array(augmentations_confidences_attack)[:, i],torch.stack(augmentations_success_attack).numpy()[:, i] ,torch.stack(augmentations_success).numpy()[:, i]))
                        for i in range(batch_size)]

                   merged_results_attack_o = [tuple(zip(np.array(augmentation_confidences_attack)[:, i], torch.stack(augmentation_success_attack).numpy()[:, i] , torch.stack(augmentation_success).numpy()[:, i]))
                        for i in range(batch_size)] 




                   sorted_results_o = [sorted(image_results, key=lambda x: x[0], reverse=False) for image_results in merged_results_o]

                   sorted_results_a = [sorted(image_results, key=lambda x: x[0], reverse=False) for image_results in merged_results_a]


                   sorted_results_attack_o = [sorted(image_results, key=lambda x: x[0], reverse=False) for image_results in merged_results_attack_o]

                   sorted_results_attack_a = [sorted(image_results, key=lambda x: x[0], reverse=False) for image_results in merged_results_attack_a]


                   #filter
                   filtered_data_o = [[tuple for tuple in sublist if tuple[-1] == 0] for sublist in sorted_results_attack_o]
                   filtered_data_a = [[tuple for tuple in sublist if tuple[-1] == 0] for sublist in sorted_results_attack_a]



                   best_result = 0
                   random_result = 0 
                   for sublist in sorted_results_o:
                        # Increment the best result based on the first item's binary value
                        best_result += sublist[0][1]
                        # Calculate the sum of random items in each sublist
                        random_result += random.choice(sublist)[1]
                   row[0]+=random_result
                   row[1]+=best_result

                   best_result = 0
                   random_result = 0 
                   for sublist in filtered_data_o:
                        # Increment the best result based on the first item's binary value
                        if len(sublist)>0:
                            best_result += sublist[0][1]
                            # Calculate the sum of random items in each sublist
                            random_result += random.choice(sublist)[1]
                   row[2]+=random_result
                   row[3]+=best_result



                   best_result = 0
                   random_result = 0 
                   for sublist in sorted_results_a:
                        best_result += sublist[0][1]
                        random_result += random.choice(sublist)[1]


                   row[4]+=random_result
                   row[5]+=best_result


                   best_result = 0
                   random_result = 0 
                   for sublist in filtered_data_a:
                     if len(sublist)>0:
                        best_result += sublist[0][1]
                        random_result += random.choice(sublist)[1]


                   row[6]+=random_result
                   row[7]+=best_result

                   print(row) 
                   del images, labels
                   torch.cuda.empty_cache()


               results.append(row)
               print(results)
               del g
               with open('imagenet_Algorithm_components_results/'+name_v+'_'+name_g+'_'+str(e)+'.pkl', 'wb') as f:
                 pickle.dump(results, f)

______________________
Victim: vit_b_16
12.75
g: efficientnet_b0
0


100%|██████████| 200/200 [24:29<00:00,  7.35s/it]


[3, 10, 21, 36, 16, 21, 16, 34]
1


100%|██████████| 200/200 [24:29<00:00,  7.35s/it]


[9, 25, 45, 76, 23, 41, 25, 67]
2


100%|██████████| 200/200 [24:04<00:00,  7.22s/it]


[15, 38, 64, 108, 37, 65, 38, 97]
3


 34%|███▍      | 68/200 [08:05<14:36,  6.64s/it]

In [ ]:
for i , name_v in enumerate(models):
       
        S = models.copy()
        S.pop(i)        
        for j , name_g in enumerate(S):
            if j==3:
                 with open('imagenet_Algorithm_components_results/'+name_v+'_'+name_g+'_'+str(e)+'.pkl', 'rb') as f:
                                 results = pickle.load(f)
                                 table_e+=results

    table_e = [[item / 1000 for item in sublist] for sublist in table_e]

    with open('imagenet_Algorithm_components_results/tables/table_'+str(e)+'.pkl', 'wb') as f:
                     pickle.dump(table_e, f)

In [ ]:
import pandas as pd

columns = ['s1_random', 's1_rank', 's1_filter_random', 's1_filter_rank','s2_random', 's2_rank', 's2_filter_random', 's2_filter_rank']

df = pd.DataFrame(table_e, columns=columns)
print(df)